In [ ]:
import numpy as np

from __init__ import *
from LIF_CORTEX import LIF as Cortex
from LIF_STRIATUM import LIF as Striatum
from LIF_GPe import LIF as Gpe
from LIF_SNr import LIF as Snr

import random as rand
import math as m
import matplotlib.pyplot as plt

class Network:
    def __init__(self, num_ctx, num_str, num_gpe, num_snr):
        self.numCtx = num_ctx
        self.neurons = [Cortex() for i in range(num_neurons)]
        self.numStr = num_str
        self.neurons = [Striatum() for i in range(num_neurons)]
        self.numGpe = num_gpe
        self.neurons = [Gpe() for i in range(num_neurons)]
        self.numSnr = num_snr
        self.neurons = [Snr() for i in range(num_neurons)]

        self.generate_weights()

    def generate_weights(self): ## w1:ctx-str w2:str-snr w3:str-gpe w4:gpe-snr w5:snr-ctx
        self.weights = np.random.rand(self.numNeurons, self.numNeurons)
        self.weights[:, self.Ne:] *= -1
        self.weights[:, :self.Ne] *= 0.5

    def fire(self):
        time = 1000
        dt = 0.5

        firings = []
        for t in range(time):
            I = np.concatenate((np.random.normal(1, 1, self.Ne) * 5, np.random.normal(1, 1, self.Ni) * 2), axis=0)
            fired = [i for i in range(self.Ne + self.Ni) if self.neurons[i].v >= 30]
            len_fired = len(fired)
            len_firings = len(firings)

            if len_fired == 0: 
                pass 
            elif len_firings == 0:
                firings = [[neuronNumber, t] for neuronNumber in fired]
            else:
                fire_time = [[neuronNumber, t] for neuronNumber in fired]
                firings = np.concatenate((firings, fire_time), axis=0)

            # update U and V to the fired ones
            for k in range(len(fired)):
                for i in range(m.ceil(1/dt)):
                    self.neurons[fired[k]].step(dt, I[fired[k]], 1)

            # update I
            I += np.sum(self.weights[:, fired], axis=1)

            for k in range(self.numNeurons):
                for i in range(m.ceil(1/dt)):
                    self.neurons[k].step(dt, I[k], 1)

        return firings 



n = Network(1000)
firings = n.fire()

y = [firings[i][0] for i in range(len(firings))]  # neurons
x = [firings[i][1] for i in range(len(firings))]  # time

plt.title("Spikes in a SNN")
plt.ylabel("Neurons")
plt.xlabel("Time [ms]")
plt.scatter(x, y, s=0.1, color='black')
plt.show()